In [1]:
%pip install -q dagshub mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.0/262.0 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 80.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 75.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.8/797.8 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -q sentence-transformers

In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import mean_squared_error
from sentence_transformers import SentenceTransformer
import time
import mlflow
import dagshub
dagshub.init(repo_owner='reddyrohith49471', repo_name='amazon-ml-2025', mlflow=True)

Initialized MLflow to track repo "reddyrohith49471/amazon-ml-2025"

Repository reddyrohith49471/amazon-ml-2025 initialized!

In [19]:
train = pd.read_csv("/kaggle/input/trainml/train.csv")

In [20]:
train.shape

(75000, 19)

In [21]:
train.head(1)

,catalog_content,image_link,price,has_item_name,has_value,has_unit,Item Name,Value,unit,num_bullets,bullet_1,bullet_2,bullet_3,bullet_4,bullet_5,combined_text,price_bin,price_clipped,price_log
0,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,True,True,True,"La Victoria Green Taco Sauce Mild, 12 Ounce (P...",72.0,Fl Oz,0,NaN,NaN,NaN,NaN,NaN,"La Victoria Green Taco Sauce Mild, 12 Ounce (P...",$0-$200,4.89,1.773256


In [22]:
X = train[["combined_text","Value"]]
Y = train["price_log"]

In [23]:
X_train, X_test, Y_train_price_log, Y_test_price_log, Y_train_price, Y_test_price = train_test_split(X,
                                                          train["price_log"],
                                                          train["price"],
                                                          test_size=0.2,
                                                          random_state=42)

In [24]:
print(X_train.shape)
print(X_test.shape)
print(Y_train_price_log.shape)
print(Y_test_price_log.shape)
print(Y_train_price.shape)
print(Y_test_price.shape)


(60000, 2)
(15000, 2)
(60000,)
(15000,)
(60000,)
(15000,)


In [ ]:
text_col = "combined_text"
num = ["Value"]
preprocessor = ColumnTransformer(
    transformers=[
        ("text",TfidfVectorizer(
            max_features=5000,
            ngram_range=(1,2),
            min_df=5,
            stop_words="english"
        ),text_col),
        ("num",StandardScaler(),num)
    ]
)

In [25]:
def decode_price(price_log_pred, upper_limit=200):
    price_clipped_pred = np.expm1(price_log_pred)
    price_pred = np.clip(price_clipped_pred, None, upper_limit)
    return price_pred


In [26]:
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0
    return np.mean(diff) * 100

In [ ]:
def run_experiment(model,run_name):
    with mlflow.start_run(run_name=run_name):
        #Train
        model.fit(X_train,Y_train_price_log)

        #Predict
        y_pred_log = model.predict(X_test)
        y_pred_price = decode_price(y_pred_log,upper_limit=200)

        #Metrics
        mse = mean_squared_error(y_pred_price,Y_test_price)
        smape_score = smape(y_pred_price,Y_test_price)
        rmse = np.sqrt(mse)

        #Logging Params
        mlflow.log_param("model", model.named_steps["regressor"].__class__.__name__)
        mlflow.log_param("text_column","combined_text")
        mlflow.log_param("numeric_column","Value")
        mlflow.log_param("max_features",5000)
        mlflow.log_param("ngram_range","(1,2)")

        #Logging Metrics
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("smape", smape_score)

        #Logging Model
        mlflow.sklearn.log_model(model,"model")
    scores = {"MSE:":mse,
             "RMSE:":rmse,
             "SMAPE:":smape_score}
    return scores
        

In [ ]:
model = Pipeline([
    ("preprocessor",preprocessor),
    ("regressor",LinearRegression())
])
run_experiment(model,"Linear_Regression_Baseline")

In [ ]:
model = Pipeline([
    ("preprocessor",preprocessor),
    ("regressor",Ridge(alpha=1))
])
run_experiment(model,"Ridge_alpha_1")

In [ ]:
model = Pipeline([
    ("preprocessor",preprocessor),
    ("regressor",Ridge(alpha=5))
])
run_experiment(model,"Ridge_alpha_5")

In [ ]:
model = Pipeline([
    ("preprocessor",preprocessor),
    ("regressor",Ridge(alpha=10))
])
run_experiment(model,"Ridge_alpha_10")

In [ ]:
def run_experiment(model,run_name):
    with mlflow.start_run(run_name=run_name):
        #Train
        model.fit(X_train,Y_train_price_log)

        #Predict
        y_pred_log = model.predict(X_test)
        y_pred_price = decode_price(y_pred_log,upper_limit=200)

        #Metrics
        mse = mean_squared_error(y_pred_price,Y_test_price)
        smape_score = smape(y_pred_price,Y_test_price)
        rmse = np.sqrt(mse)

        #Logging Params
        mlflow.log_param("model", model.named_steps["regressor"].__class__.__name__)
        mlflow.log_param("text_column","combined_text")
        mlflow.log_param("numeric_column","Value")
        mlflow.log_param("max_features",5000)
        mlflow.log_param("ngram_range","(1,1)")

        #Logging Metrics
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("smape", smape_score)

        #Logging Model
        mlflow.sklearn.log_model(model,"model")
    scores = {"MSE:":mse,
             "RMSE:":rmse,
             "SMAPE:":smape_score}
    return scores
        

In [ ]:
text_col = "combined_text"
num = ["Value"]
preprocessor = ColumnTransformer(
    transformers=[
        ("text",TfidfVectorizer(
            max_features=5000,
            ngram_range=(1,1),
            min_df=5,
            stop_words="english"
        ),text_col),
        ("num",StandardScaler(),num)
    ]
)

In [ ]:
model = Pipeline([
    ("preprocessor",preprocessor),
    ("regressor",LinearRegression())
])
run_experiment(model,"Linear_Regression_Baseline")

In [ ]:
model = Pipeline([
    ("preprocessor",preprocessor),
    ("regressor",Ridge(alpha=1))
])
run_experiment(model,"Ridge_alpha_1")

In [ ]:
model = Pipeline([
    ("preprocessor",preprocessor),
    ("regressor",Ridge(alpha=5))
])
run_experiment(model,"Ridge_alpha_5")

In [ ]:
model = Pipeline([
    ("preprocessor",preprocessor),
    ("regressor",Ridge(alpha=10))
])
run_experiment(model,"Ridge_alpha_10")

In [28]:
def run_experiment(model,run_name):
    with mlflow.start_run(run_name=run_name):
        #Train
        model.fit(X_train,Y_train_price_log)

        #Predict
        y_pred_log = model.predict(X_test)
        y_pred_price = decode_price(y_pred_log,upper_limit=200)

        #Metrics
        mse = mean_squared_error(y_pred_price,Y_test_price)
        smape_score = smape(y_pred_price,Y_test_price)
        rmse = np.sqrt(mse)

        #Logging Params
        mlflow.log_param("model", model.named_steps["regressor"].__class__.__name__)
        mlflow.log_param("text_column","combined_text")
        mlflow.log_param("numeric_column","Value")
        mlflow.log_param("max_features",10000)
        mlflow.log_param("ngram_range","(1,2)")

        #Logging Metrics
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("smape", smape_score)

        #Logging Model
        mlflow.sklearn.log_model(model,"model")
    scores = {"MSE:":mse,
             "RMSE:":rmse,
             "SMAPE:":smape_score}
    return scores
        

In [29]:
text_col = "combined_text"
num = ["Value"]
preprocessor = ColumnTransformer(
    transformers=[
        ("text",TfidfVectorizer(
            max_features=10000,
            ngram_range=(1,2),
            min_df=5,
            stop_words="english"
        ),text_col),
        ("num",StandardScaler(),num)
    ]
)

In [ ]:
model = Pipeline([
    ("preprocessor",preprocessor),
    ("regressor",LinearRegression())
])
run_experiment(model,"Linear_Regression_Baseline")

In [ ]:
model = Pipeline([
    ("preprocessor",preprocessor),
    ("regressor",Ridge(alpha=1))
])
run_experiment(model,"Ridge_alpha_1")

In [ ]:
model = Pipeline([
    ("preprocessor",preprocessor),
    ("regressor",Ridge(alpha=5))
])
run_experiment(model,"Ridge_alpha_5")

In [ ]:
model = Pipeline([
    ("preprocessor",preprocessor),
    ("regressor",Ridge(alpha=10))
])
run_experiment(model,"Ridge_alpha_10")

In [ ]:
to_dense = FunctionTransformer(lambda x: x.toarray(), accept_sparse=True)
model = Pipeline([
    ("preprocessor", preprocessor),
    ("to_dense", to_dense),
    ("regressor", HistGradientBoostingRegressor())
])
run_experiment(model,"Hist_GB_Regressor")

In [30]:
model = Pipeline([
    ("preprocessor",preprocessor),
    ("regressor",XGBRegressor())
])
run_experiment(model,"TFIDF_XGBR")

2026/02/11 13:34:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run TFIDF_XGBR at: https://dagshub.com/reddyrohith49471/amazon-ml-2025.mlflow/#/experiments/0/runs/4f16b24182764ba687bf9dc1bf44de83
🧪 View experiment at: https://dagshub.com/reddyrohith49471/amazon-ml-2025.mlflow/#/experiments/0


{'MSE:': 1204.314411197203,
 'RMSE:': np.float64(34.70323344008744),
 'SMAPE:': np.float64(56.18232813580845)}

In [12]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
def embed_text(text_series):
    return embedder.encode(
        text_series.tolist(),
        show_progress_bar=False,
        convert_to_numpy=True
    )


In [14]:
def run_experiment(model, run_name):
    with mlflow.start_run(run_name=run_name):

        # Train
        model.fit(X_train, Y_train_price_log)

        # Predict
        y_pred_log = model.predict(X_test)
        y_pred_price = decode_price(y_pred_log, upper_limit=200)

        # Metrics
        mse = mean_squared_error(Y_test_price, y_pred_price)
        rmse = np.sqrt(mse)
        smape_score = smape(Y_test_price, y_pred_price)

        # Logging Params
        mlflow.log_param("model", model.named_steps["regressor"].__class__.__name__)
        mlflow.log_param("text_column", "combined_text")
        mlflow.log_param("numeric_column", "Value")

        # Logging Metrics
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("smape", smape_score)

        # Logging Model
        mlflow.sklearn.log_model(model, "model")

    scores = {
        "MSE": mse,
        "RMSE": rmse,
        "SMAPE": smape_score
    }

    return scores


In [16]:
text_col = "combined_text"
num = ["Value"]
preprocessor = ColumnTransformer(
    transformers=[
        ("text",FunctionTransformer(embed_text,validate=False),text_col),
        ("num",StandardScaler(),num)
    ]
)


In [17]:
model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", LinearRegression())
])

run_experiment(model,"transformer_LR")

2026/02/11 12:26:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
2026/02/11 12:26:33 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.23.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torchvision==0.23.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


🏃 View run transformer_LR at: https://dagshub.com/reddyrohith49471/amazon-ml-2025.mlflow/#/experiments/0/runs/65577c37a82d4e8e943f68c1765fa1a8
🧪 View experiment at: https://dagshub.com/reddyrohith49471/amazon-ml-2025.mlflow/#/experiments/0


{'MSE': 1403.394335270968,
 'RMSE': np.float64(37.461905120681834),
 'SMAPE': np.float64(64.55172211200197)}

In [18]:
model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", Ridge(alpha=1.0))
])

run_experiment(model,"transformer_Ridge_alpha1")

2026/02/11 12:58:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
2026/02/11 12:59:02 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.23.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torchvision==0.23.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


🏃 View run transformer_Ridge_alpha1 at: https://dagshub.com/reddyrohith49471/amazon-ml-2025.mlflow/#/experiments/0/runs/28a75f8a94a6452397ca7af34dbed79a
🧪 View experiment at: https://dagshub.com/reddyrohith49471/amazon-ml-2025.mlflow/#/experiments/0


{'MSE': 1410.2189868203318,
 'RMSE': np.float64(37.55288253676849),
 'SMAPE': np.float64(64.68962491573821)}

In [19]:
model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", Ridge(alpha=5.0))
])

run_experiment(model,"transformer_Ridge_alpha5")

2026/02/11 13:00:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
2026/02/11 13:00:31 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.23.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torchvision==0.23.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


🏃 View run transformer_Ridge_alpha5 at: https://dagshub.com/reddyrohith49471/amazon-ml-2025.mlflow/#/experiments/0/runs/1fad097fe87347eeb158bcadade29b05
🧪 View experiment at: https://dagshub.com/reddyrohith49471/amazon-ml-2025.mlflow/#/experiments/0


{'MSE': 1421.676723734817,
 'RMSE': np.float64(37.70512861315841),
 'SMAPE': np.float64(64.90655137851279)}

In [20]:
model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", Ridge(alpha=10.0))
])

run_experiment(model,"transformer_Ridge_alpha10")

2026/02/11 13:01:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
2026/02/11 13:02:06 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.23.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torchvision==0.23.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


🏃 View run transformer_Ridge_alpha10 at: https://dagshub.com/reddyrohith49471/amazon-ml-2025.mlflow/#/experiments/0/runs/36e844129966498dbddb2627f82630a6
🧪 View experiment at: https://dagshub.com/reddyrohith49471/amazon-ml-2025.mlflow/#/experiments/0


{'MSE': 1431.1983765707234,
 'RMSE': np.float64(37.83118259545587),
 'SMAPE': np.float64(65.1139508534865)}

In [18]:
model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor())
])

run_experiment(model,"transformer_XGBR")

2026/02/11 13:23:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
2026/02/11 13:24:24 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.23.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torchvision==0.23.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


🏃 View run transformer_XGBR at: https://dagshub.com/reddyrohith49471/amazon-ml-2025.mlflow/#/experiments/0/runs/706527b02c9a4964b4d8a3f1b7933fee
🧪 View experiment at: https://dagshub.com/reddyrohith49471/amazon-ml-2025.mlflow/#/experiments/0


{'MSE': 1265.5586313299561,
 'RMSE': np.float64(35.574690881720336),
 'SMAPE': np.float64(59.777803369221395)}

In [33]:
import mlflow.pyfunc

model = mlflow.pyfunc.load_model(
    "models:/XGBoost/Staging"
)


In [36]:
sample = pd.DataFrame({
    "combined_text": ["Organic tomato sauce 500g"],
    "Value": [500]
})


In [38]:
start = time.time()
pred_log = model.predict(sample)
pred_price = np.expm1(pred_log)
end = time.time()

latency_ms = (end - start) * 1000

print("Predicted price:", pred_price[0])
print("Latency:", latency_ms, "ms")


Predicted price: 28.201097
Latency: 6.475687026977539 ms


In [39]:
latencies = []
for i in range(100):
    start = time.time()
    model.predict(sample)
    end = time.time()
    latencies.append((end - start) * 1000)

latencies = np.array(latencies)
print("Average latency:", latencies.mean(), "ms")
print("Median latency:", np.median(latencies), "ms")
print("P95 latency:", np.percentile(latencies, 95), "ms")
print("Max latency:", latencies.max(), "ms")


Average latency: 3.4674906730651855 ms
Median latency: 3.3789873123168945 ms
P95 latency: 3.7967681884765625 ms
Max latency: 6.989717483520508 ms
